In [ ]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
emotion = load_dataset('dair-ai/emotion')
emotion.set_format(type='pandas')

emotion

In [ ]:
train = emotion['train'][:]

train.head()

In [ ]:
classes = emotion['train'].features['label'].names

classes

In [ ]:
train['label_name'] = train['label'].apply(lambda x: classes[x])

train.head()

### Data Analysis

In [ ]:
label_counts = train['label_name'].value_counts(ascending=True)
label_counts.plot.bar()
plt.title('Frequency of Classes')
plt.show()

In [ ]:
train['Words per Tweet'] = train['text'].str.split().apply(len)

train.head()

In [ ]:
train.boxplot('Words per Tweet', by='label_name')

### Text to Token Conversion

In [ ]:
model_ckpt = 'distilbert-base-uncased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
# Tokenization Example
text = "I love Machine Learning!. Tokenization is awesome"
encoded_text = tokenizer(text)

encoded_text

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)

tokens

In [ ]:
# Total vocab size (trained data token size) and token size limit for the model
tokenizer.vocab_size, tokenizer.model_max_length

### Tokenization of the Emotion Data

In [ ]:
emotion.reset_format()

emotion

In [ ]:
def tokenize(batch):
    temp = tokenizer(batch['text'], padding = True, truncation = True)
    return temp


tokenize(emotion['train'][:5])

In [ ]:
emotions_encoded = emotion.map(tokenize, batched = True, batch_size = None)

emotions_encoded

### Model Building

In [ ]:
text

In [ ]:
inputs = tokenizer(text, return_tensors = 'pt')

inputs

In [ ]:
model = AutoModel.from_pretrained(model_ckpt)

model

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

outputs

In [ ]:
last_hidden_states = outputs.last_hidden_state

last_hidden_states

In [ ]:
last_hidden_states.shape

### Fine Tuning Transformers

In [ ]:
num_labels = len(classes)

num_labels

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = num_labels).to(device)

model

In [ ]:
batch_size = 64
model_name = 'distilbert-finetuned-emotion'

In [ ]:
training_args = TrainingArguments(output_dir = "./model/" + model_name,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_eval_batch_size = batch_size,
                                  per_device_train_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  eval_strategy = 'epoch',
                                  disable_tqdm = False)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average = 'weighted')
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1 Score": f1}

In [ ]:
trainer = Trainer(model = model,
                  args = training_args,
                  compute_metrics = compute_metrics,
                  train_dataset = emotions_encoded['train'],
                  eval_dataset = emotions_encoded['validation'],
                  tokenizer = tokenizer)

In [ ]:
trainer.train()

In [ ]:
pred_outputs = trainer.predict(emotions_encoded['test'])
pred_outputs.metrics

In [ ]:
pred_outputs

In [ ]:
y_preds = np.argmax(pred_outputs.predictions, axis = 1)

y_preds

In [ ]:
y_true = emotions_encoded['test'][:]['label']

y_true

In [ ]:
print(classes)
print(classification_report(y_true, y_preds))

In [ ]:
label_counts

In [ ]:
text = 'I hate you'
if torch.backends.mps.is_available():
    device = torch.device("mps")
input_encoded = tokenizer(text, return_tensors = 'pt').to(device)
with torch.no_grad():
    outputs = model(**input_encoded)

outputs

In [ ]:
logits = outputs.logits
pred = torch.argmax(logits, dim = 1).item()

pred, classes[pred]